# Speed test notebook

In [1]:
#!/usr/bin/env python
# coding: utf-8
# (c) Charles Le Losq, Clément Ferraina 2023-2024
# see embedded licence file

import numpy as np

import gpvisc

2024-07-04 09:57:03.447254: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-04 09:57:03.470242: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-04 09:57:03.861143: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Creating dummy data for the query

In [2]:
Inputs_ = gpvisc.generate_query_single(sio2=100.0,
                                       composition_mole=True,
                                       T_init=1050., T_final=2000.,
                                       nb_values=10)

tpxi_scaled = gpvisc.scale_for_gaussianprocess( 
                               Inputs_.loc[:,"T"], # temperature input
                               Inputs_.loc[:,"P"], # pressure input
                               Inputs_.loc[:,gpvisc.list_oxides()] # composition input
                               )

## Predictions with GPU

### GP model

In [3]:
gp_model, likelihood = gpvisc.load_gp_model(model_number=1, device="cuda")

In [4]:
%%timeit
gpvisc.predict(tpxi_scaled, gp_model, likelihood, device="cuda")

12.3 ms ± 407 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Greybox ANN

In [5]:
%%timeit
gpvisc.predict(tpxi_scaled, gp_model, likelihood, model_to_use="ann", device="cuda")

282 μs ± 4.48 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Predictions with CPU

### GP model

In [6]:
gp_model2, likelihood2 = gpvisc.load_gp_model(model_number=1, device="cpu")

In [7]:
%%timeit
gpvisc.predict(tpxi_scaled, gp_model2, likelihood2, device="cpu")

294 ms ± 4.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Greybox ANN

In [8]:
%%timeit
gpvisc.predict(tpxi_scaled, gp_model2, likelihood2, model_to_use="ann", device="cpu")

524 μs ± 472 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
